# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [35]:
pip install azureml-sdk==1.59.0

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.59.0
Summary: Used to build and run machine learning workflows upon the     Azure Machine Learning service.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/automl-env/lib/python3.9/site-packages
Requires: azureml-core, azureml-dataset-runtime, azureml-pipeline, azureml-train-automl-client, azureml-train-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install 'azureml-sdk[notebooks]'

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install azureml-sdk[automl]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import pandas as pd
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.model import InferenceConfig
import logging
import csv
import numpy as np
import pkg_resources
import azureml.core
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core import Workspace, Experiment, Modelfrom azureml.core.compute import ComputeTarget, AmlComputefrom azureml.core.compute_target import ComputeTargetExceptionfrom azureml.core import Datasetimport sysfrom azureml.core.model import InferenceConfigfrom azureml.core.webservice import AciWebservice, Webservicefrom azureml.core.model import Modelfrom azureml.automl.core.shared import constant


## 

### Overview
TODO: In this markdown cell, give an overview of the  you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the  needs to be external.

In [40]:
from azureml.core import Workspace, Experiment

project_folder = '.'

Exp_name="Proj3AUTOML"
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=Exp_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-281073
Azure region: westeurope
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-281073


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [41]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "ClusterML"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [42]:
from train import clean_data
import pandas as pd

webpath = "https://raw.githubusercontent.com/Jbjcool1997/proj3ML/refs/heads/main/heart_failure_clinical_records_dataset.csv"
df =pd.read_csv(webpath)
df.head()


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [43]:
Label_column = 'DEATH_EVENT'

In [44]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "proj3data_ML"
description_text = "heartfailure"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Jbjcool1997/proj3ML/refs/heads/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time
count,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000
mean,60.833893,581.839465,38.083612,263358.029264,1.39388,136.625418,130.260870
std,11.894809,970.287881,11.834841,97804.236869,1.03451,4.412477,77.614208
min,40.000000,23.000000,14.000000,25100.000000,0.50000,113.000000,4.000000
25%,51.000000,116.500000,30.000000,212500.000000,0.90000,134.000000,73.000000
50%,60.000000,250.000000,38.000000,262000.000000,1.10000,137.000000,115.000000
75%,70.000000,582.000000,45.000000,303500.000000,1.40000,140.000000,203.000000
max,95.000000,7861.000000,80.000000,850000.000000,9.40000,148.000000,285.000000


In [ ]:
from azureml.train.automl import AutoMLConfig


automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric_name" : 'Accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name=Label_column,   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                              enable_onnx_compatible_models=True,
                             **automl_settings
                            )

In [ ]:
from azureml.pipeline.core import PipelineData, TrainingOutput

#ds = ws.get_default_datastore()
#metrics_output_name = 'metrics_output'
#best_model_output_name = 'best_model_output'

#metrics_data = PipelineData(name='metrics_data',
#                           datastore=ds,
#                           pipeline_output_name=metrics_output_name,
#                           training_output=TrainingOutput(type='Metrics'))
#model_data = PipelineData(name='model_data',
#                           datastore=ds,
#                           pipeline_output_name=best_model_output_name,
#                           training_output=TrainingOutput(type='Model'))

In [ ]:
#automl_step = AutoMLStep(
#    name='automl_module',
#    automl_config=automl_config,
#    config = automl_config,
#    outputs=[metrics_data, model_data],
#    allow_reuse=True)

In [ ]:
#from azureml.pipeline.core import Pipeline
#pipeline = Pipeline(
#    description="pipeline_with_automlstep",
#    workspace=ws, 
#    steps=[automl_step])

In [ ]:
from azureml.core.experiment import Experiment
automl_run = exp.submit(config=automl_config, show_output=True)

Created step automl_module [8efbe5d3][ada023dd-0f00-413b-8ece-583f67cef2e3], (This step will run and generate new outputs)
Submitted PipelineRun 13334919-818e-4e51-9ce1-88a345a885be
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/13334919-818e-4e51-9ce1-88a345a885be?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-281073/workspaces/quick-starts-ws-281073&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [ ]:
from collections.abc import Mapping
from azureml.widgets import RunDetails

RunDetails(automl_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported


In [ ]:
automl_run.wait_for_completion()

PipelineRunId: 13334919-818e-4e51-9ce1-88a345a885be
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/13334919-818e-4e51-9ce1-88a345a885be?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-281073/workspaces/quick-starts-ws-281073&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 3aa522df-c1eb-4569-ba2b-4aa3dfef2971
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/3aa522df-c1eb-4569-ba2b-4aa3dfef2971?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-281073/workspaces/quick-starts-ws-281073&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: NotStarted
StepRun( automl_module ) Status: Running


In [ ]:
###NEW get the best model

best_automl_run, best_automl_model = automl_run.get_output(return_onnx_model=True)
#best_run.register_model(model_name = 'automl_best_model.pkl', model_path='outputs/')

print(best_automl_run)
print(best_automl_model)

#saving model in onnx format in outputs directory
OnnxConverter.save_onnx_model(best_automl_model, file_path="./outputs/automl_model.onnx")

In [ ]:
best_run_properties = best_automl_run.get_properties()
print(best_run_properties)

In [ ]:
#Register and save the best model

best_automl_run.download_file('./outputs/model.pkl', './outputs/capstone_automl_model.pkl')

model = best_automl_run.register_model(model_name='automl_best_model', model_path='./outputs/model.pkl')

print(model)

#run.complete()

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
##### MODEL DEPLOYMENT

#Obtaining CODA evirionment data used in run and saving it to directory
best_automl_run.download_file(constants.CONDA_ENV_FILE_PATH, 'heart_dependencies.yml')

environment = Environment.from_conda_specification('heart_dependencies', 'heart_dependencies.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

service_name = 'proj3_best_automl_deploy'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config,
                          overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.state)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
service.get_logs()

In [ ]:
print(service.scoring_uri)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
import json

data = {"data":
        [
          {
            "age": 75,
            "anaemia": 0,
            "creatinine_phosphokinase": 582,
            "diabetes": 0,
            "ejection_fraction": 20,
            "high_blood_pressure": 1,
            "platelets": 265000,
            "serum_creatinine": 1.9,
            "serum_sodium": 130,
            "sex": 1,
            "smoking": 0,
            "time": 4
          },
          {
            "age": 25,
            "anaemia": 1,
            "creatinine_phosphokinase": 425,
            "diabetes": 1,
            "ejection_fraction": 50,
            "high_blood_pressure": 1,
            "platelets": 300000,
            "serum_creatinine": 0.9,
            "serum_sodium": 150,
            "sex": 0,
            "smoking": 1,
            "time": 100
          }
      ]
    }

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print(service.get_logs())

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
